In [ ]:

!pip install pandas langchain openai
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install -U langchain
!pip install --upgrade langchain
!pip install streamlit -q
!sudo apt-get update
!pip install streamlit langchain pandas python-dotenv
!pip install faiss-cpu

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
%%writefile kfc_chatbot.py
import os
import pandas as pd
import json
import re
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RouterChain
from tenacity import retry, stop_after_attempt, wait_random_exponential
import openai
import streamlit as st
import difflib

openai.api_key = "API KEY"
deployment_name = "gpt-35-turbo"
azure_endpoint = "END POINT"
api_version = "2024-12-01-preview"


llm = AzureChatOpenAI(
    openai_api_version=api_version,
    deployment_name=deployment_name,
    model="gpt-35-turbo",
    temperature=0,
    api_key=openai.api_key,
    azure_endpoint=azure_endpoint,
)

# Load menu CSV
df = pd.read_csv("kfc_menu.csv")
texts = df['Products'].astype(str).tolist()
descriptions = df['Description'].fillna("").astype(str).tolist()
prices = df['price'].astype(float).tolist()  # Ensure price is float

# Format the menu display
menu_lines = [f" - {item} (₹{price})" for item, price in zip(texts, prices)]
menu_display = "\n".join(menu_lines)

# Prompt templates
price_prompt = PromptTemplate.from_template("""
You are a helpful assistant that returns the price of a KFC menu item.
Menu Items: {texts}
Prices: {prices}
Query: {input}
""")

item_prompt = PromptTemplate.from_template("""
You are a KFC assistant. Show the menu when the user asks for it.
Here's the menu:
{menu_display}
User Query: {input}
""")

description_prompt = PromptTemplate.from_template("""
You are a helpful assistant that provides a description of KFC menu items.
Items: {texts}
Descriptions: {descriptions}
Query: {input}
""")

order_prompt = PromptTemplate.from_template("""
You are a KFC assistant helping to manage an order.
The current order is:
{order_summary}

User Query: {input}
""")

router_template = PromptTemplate.from_template("""
Classify the following user query into one of these categories:
- price: if asking about price
- item: if asking for menu or list of items
- description: if asking what an item is or what's in it
- order: if the user is placing an order or modifying it

Return in JSON format, and ONLY JSON format:
{
    "destination": "price" or "item" or "description" or "order",
    "next_inputs": {"input": "{input}"}
}

Query: {input}
""")

# Router chain
router_llm_chain = LLMChain(
    llm=llm,
    prompt=router_template,
)

# Router parser
def parse_router_output(output_text):
    try:
        json_match = re.search(r'\{.*\}', output_text, re.DOTALL)
        if json_match:
            json_string = json_match.group(0)
            routed_data = json.loads(json_string)
            if "destination" in routed_data and "next_inputs" in routed_data and "input" in routed_data["next_inputs"]:
                return routed_data
    except (json.JSONDecodeError, KeyError, TypeError):
        pass
    return None

# Order handling
if 'order' not in st.session_state:
    st.session_state.order = []

def add_to_order(item, price):
    # Check for exact match first
    for existing_item in st.session_state.order:
        if existing_item["item"].lower() == item.lower():
            return  # Prevent duplicates (exact match)

    # Fallback to checking for similar items with a lower threshold for "close matches"
    match = difflib.get_close_matches(item.lower(), [p.lower() for p in texts], n=1, cutoff=0.7)
    if match:
        match_name = next(p for p in texts if p.lower() == match[0])
        price = prices[texts.index(match_name)]
        st.session_state.order.append({"item": match_name, "price": price})
        st.write(f"🛒 Added to order: {match_name} (₹{price})")
    else:
        st.write(f"🤖 Bot: I couldn't find a match for {item}. Please try again.")

def show_order_summary():
    if not st.session_state.order:
        return "🛒 Your order is empty."

    summary = "📝 Your order:\n"
    total = 0
    for item in st.session_state.order:
        summary += f"- {item['item']} (₹{item['price']})\n"
        total += item['price']
    summary += f"\nTotal: ₹{total}"
    return summary

def remove_from_order(item):
    st.session_state.order = [o for o in st.session_state.order if o["item"].lower() != item.lower()]

def save_order_to_file():
    if not st.session_state.order:
        return None
    df_order = pd.DataFrame(st.session_state.order)
    df_order["price"] = df_order["price"].astype(float)
    df_order.loc[len(df_order.index)] = ["Total", df_order["price"].sum()]
    file_path = "order_summary.csv"
    df_order.to_csv(file_path, index=False)
    return file_path

# Streamlit UI
st.title("🍗 KFC Chatbot")
st.write("Welcome to KFC Chatbot! Ask about menu, prices, descriptions, or place an order.")

query = st.text_input("🧑 You:", "")

if query:
    if query.lower() in ["exit", "quit", "bye"]:
        st.write("👋 Bye! Enjoy your KFC meal!")
    elif "summary" in query.lower():
        st.write(show_order_summary())
    elif "place order" in query.lower() or "place the order" in query.lower():
        if not st.session_state.order:
            st.write("🛒 Your order is empty. Add items before placing the order.")
        else:
            path = save_order_to_file()
            st.write("✅ Your order has been saved!")
            with open(path, "rb") as file:
                st.download_button(label="Download order summary", data=file, file_name="order_summary.csv", mime="text/csv")
    elif "remove" in query.lower() or "delete" in query.lower():
        words = query.split()
        for word in words:
            match = difflib.get_close_matches(word, [p.lower() for p in texts], n=1, cutoff=0.6)
            if match:
                match_name = next(p for p in texts if p.lower() == match[0])
                remove_from_order(match_name)
                st.write(f"🛒 Removed from order: {match_name}")
    elif "order" in query.lower():
        items = query.split(',')
        for item in items:
            item = item.strip()
            add_to_order(item, None)
    else:
        @retry(stop=stop_after_attempt(3), wait=wait_random_exponential(multiplier=1, max=10))
        def invoke_with_retry(input_data):
            return router_llm_chain.invoke(input_data)

        try:
            router_output = invoke_with_retry({"input": query})
            routed_data = parse_router_output(router_output['text'])

            if routed_data:
                destination = routed_data['destination']
                next_input = routed_data['next_inputs']['input']

                if destination == "price":
                    response = LLMChain(llm=llm, prompt=price_prompt).invoke({"input": next_input, "texts": texts, "prices": prices})
                elif destination == "description":
                    response = LLMChain(llm=llm, prompt=description_prompt).invoke({"input": next_input, "texts": texts, "descriptions": descriptions})
                else:
                    response = LLMChain(llm=llm, prompt=item_prompt).invoke({"input": next_input, "menu_display": menu_display})

                st.write(f"🤖 Bot:\n{response['text']}")
        except Exception:
            response = LLMChain(llm=llm, prompt=item_prompt).invoke({"input": query, "menu_display": menu_display})
            st.write(f"🤖 Bot:\n{response['text']}")

Overwriting kfc_chatbot.py


In [ ]:
!streamlit run kfc_chatbot.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.44.218:8501

⠴⠦⠧⠇your url is: https://better-horses-stare.loca.lt
/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import AzureOpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import AzureOpenAI`.

To install langchain-communi